#Upload files

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving cars.txt to cars.txt


#Exercise


1.   Install and configure kafka
2.   Create 2 topics - cars, cars2
3.   Use KafkaProducerThread to generate random data to 2 topics
4.   Create consumer to read both topics
5.   Create dict to save messages from kafka according to key topic name
6.   After 50 iterations stop reading kafka messages and print results for each topic from dict



 

#Solution

### Download and setup Kafka and Zookeeper instances

For demo purposes, the following instances are setup locally:

- Kafka (Brokers: 127.0.0.1:9092)
- Zookeeper (Node: 127.0.0.1:2181)


In [ ]:
!curl -sSOL http://packages.confluent.io/archive/5.4/confluent-community-5.4.1-2.12.tar.gz
!tar -xzf confluent-community-5.4.1-2.12.tar.gz
!pip install kafka-python

     |████████████████████████████████| 256kB 11.7MB/s 


##Run Kafka Zookeper and Server 
Using the default configurations (provided by the confluent package) for spinning up the instances.

In [ ]:
!cd confluent-5.4.1 && bin/zookeeper-server-start -daemon etc/kafka/zookeeper.properties
!cd confluent-5.4.1 && bin/kafka-server-start -daemon etc/kafka/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 10

Waiting for 10 secs until kafka and zookeeper services are up and running


##Create topic

In [ ]:
!confluent-5.4.1/bin/kafka-topics --create --zookeeper 127.0.0.1:2181 --replication-factor 1 --partitions 1 --topic cars
!confluent-5.4.1/bin/kafka-topics --create --zookeeper 127.0.0.1:2181 --replication-factor 1 --partitions 1 --topic cars2

Created topic cars.
Created topic cars2.


Describe the topic for details on the configuration

In [ ]:
!confluent-5.4.1/bin/kafka-topics --bootstrap-server 127.0.0.1:9092 --describe --topic cars
!confluent-5.4.1/bin/kafka-topics --bootstrap-server 127.0.0.1:9092 --describe --topic cars2

Topic: cars	PartitionCount: 1	ReplicationFactor: 1	Configs: segment.bytes=1073741824
	Topic: cars	Partition: 0	Leader: 0	Replicas: 0	Isr: 0
Topic: cars2	PartitionCount: 1	ReplicationFactor: 1	Configs: segment.bytes=1073741824
	Topic: cars2	Partition: 0	Leader: 0	Replicas: 0	Isr: 0


##Write event into topic - Producer

In [ ]:
import threading
import random
import socket
import sys
import time
from kafka import KafkaProducer

class KafkaProducerThread(threading.Thread):
    def __init__(self):
      threading.Thread.__init__(self)
      self.is_running = True

    def run(self):
      file = open('cars.txt', 'r')
      cars = file.readlines()
      producer = KafkaProducer(bootstrap_servers=['127.0.0.1:9092'])
      print('Kafka producer started.')
      while True:
        producer.send('cars', (cars[random.randint(5, 8)].strip('\n')+"_"+str(random.randint(1, 10))).encode('utf-8'))
        producer.send('cars', (cars[random.randint(7, 11)].strip('\n')+"_"+str(random.randint(1, 10))).encode('utf-8'))
        producer.send('cars', (cars[random.randint(0, len(cars)-1)].strip('\n')+"_"+str(random.randint(1, 10))).encode('utf-8'))
        producer.send('cars2', (cars[random.randint(0, len(cars)-1)].strip('\n')+"_"+str(random.randint(1, 10))).encode('utf-8'))
        producer.flush()
        time.sleep(0.2)
        if not self.is_running:
          break

KafkaProducerThread = KafkaProducerThread()
KafkaProducerThread.setDaemon(True)
KafkaProducerThread.start()

##Read events - Consumer

In [ ]:
from kafka import TopicPartition
from kafka import KafkaConsumer

consumer = KafkaConsumer(bootstrap_servers='localhost:9092')
consumer.assign([TopicPartition('cars', 0), TopicPartition('cars2', 0)])
cars_dict = {'cars':[], 'cars2':[]}
counter = 0

for msg in consumer:
  topic = msg.topic
  value = msg.value.decode("utf-8") 
  cars_dict[topic].append(value)
  counter += 1

  if counter > 50:
    # break infinite stream to see the results
    break
  
print(cars_dict['cars'])
print(cars_dict['cars2'])

['Honda_2', 'Kia_5', 'Land Rover_4', 'Honda_3', 'Mazda_8', 'Renault_10', 'Kia_3', 'Mazda_3', 'Mercedes-Benz_8', 'Kia_10', 'Mercedes-Benz_10', 'Audi_10', 'Kia_2', 'Land Rover_7', 'Citroën_10', 'Honda_3', 'Kia_7', 'Renault_9', 'Kia_10', 'Kia_4', 'BMW_6', 'Kia_9', 'Mazda_1', 'Subaru_1', 'Honda_10', 'Kia_3', 'Citroën_8', 'Hyundai_4', 'Land Rover_5', 'Toyota_1', 'Honda_1', 'Mercedes-Benz_5', 'Fiat_6', 'Hyundai_9', 'Mazda_3', 'Toyota_5', 'Kia_2', 'Mercedes-Benz_8', 'Nissan_10']
['Honda_8', 'Fiat_10', 'Subaru_3', 'Škoda_3', 'Porsche_8', 'Fiat_1', 'Jeep_8', 'Volkswagen_4', 'Mercedes-Benz_4', 'Honda_6', 'Porsche_8', 'Nissan_8']
